<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [ ]:
# Install spark

In [ ]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q




import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler


def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')


In [ ]:
# Start session

In [ ]:


spark = SparkSession.builder.appName("Saving and Loading a SparkML Model").getOrCreate()



In [ ]:
# Download The search term dataset from the below url
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [ ]:
# Load the csv into a spark dataframe

In [ ]:
df = spark.read.csv("searchterms.csv", header=True)

In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [ ]:
rowcount = df.count()
print(rowcount)

colcount = len(df.columns)
print(colcount)

In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [ ]:
df.show(5)

In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [ ]:
searchterm_dtype = [field.dataType for field in df.schema.fields if field.name == "searchterm"][0]
print(f"data type of 'searchterm' column: {searchterm_dtype}")

In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [ ]:
count_gaming_laptop = df.filter(df.searchterm == 'gaming laptop').count()

print(f"The term 'gaming laptop' was searched {count_gaming_laptop} times.")

In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [ ]:
from pyspark.sql import functions as F

searchterm_counts = df.groupBy("searchterm").agg(F.count("searchterm").alias("count"))

searchterm_counts.orderBy(F.desc("count")).show(5)

In [ ]:
# The pretrained sales forecasting model is available at  the below url
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [28]:
!ls /resources/labs/DB0321EN/
#!pwd
#!tar -xvf model.tar

Practise_Lab_-_Model_saving_and_loading.ipynb  searchterms.csv.1
Spark_MLOps.ipynb			       searchterms.csv.2
model.tar				       searchterms.csv.3
model.tar.gz				       searchterms.csv.4
model.tar.gz.1				       searchterms.csv.5
sales_prediction.model			       searchterms.csv.6
searchterms.csv				       searchterms.csv.7


In [37]:
from pyspark.ml.regression import LinearRegressionModel

#spark = SparkSession.builder.appName("LoadModel").getOrCreate()
model_path = "/resources/labs/DB0321EN/sales_prediction.model"
loaded_model = LinearRegressionModel.load(model_path)

loaded_model.params

[Param(parent='LinearRegression_6d5736f3dbe7', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0.'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='featuresCol', doc='features column name'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='fitIntercept', doc='whether to fit an intercept term'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='labelCol', doc='label column name'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError)'),
 Param(parent='LinearRegression_6d5736f3dbe7', name='maxIter', doc

In [38]:
print(loaded_model.coefficients)
print(loaded_model.intercept)

[6.522567861288859]
-13019.989140447298


In [39]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [40]:
df_2023 = df.filter(df.year == 2021)
print(df_2023)
df_2023.show(5)

DataFrame[day: string, month: string, year: string, searchterm: string]
+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [47]:
def predict(year):
    assembler = VectorAssembler(inputCols=["year"], outputCol="features")
    data = [[year,0]]
    columns = ["year", "sales"]
    df_ = spark.createDataFrame(data, columns)
    df__ = assembler.transform(df_).select('features','sales')
    predictions = loaded_model.transform(df__)
    predictions.select('prediction').show()

In [48]:
predict(2023)

#spark = SparkSession.builder.appName("PredictSales").getOrCreate()

#model_path = "/resources/labs/DB0321EN/sales_prediction.model"
#loaded_model = LinearRegressionModel.load(model_path)

#from pyspark.sql.functions import col
#from pyspark.ml.feature import VectorAssembler

#df_2023 = df_2023.withColumn("day", col("day").cast("int"))
#df_2023 = df_2023.withColumn("month", col("month").cast("int"))
#df_2023 = df_2023.withColumn("year", col("year").cast("int"))


#input_cols = ["day", "month", "year"]
#assembler = VectorAssembler(inputCols=input_cols, outputCol="features")
#df_2023_transformed = assembler.transform(df_2023)

#predictions_2023 = loaded_model.transform(df_2023_transformed)
#predictions_2023.select("day", "month", "year", "searchterm", "prediction").show()
#spark.stop()

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



23/10/20 20:13:04 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/10/20 20:13:04 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
